In [40]:
import spacy
from spacy import displacy
from IPython.display import display
import pandas as pd

nlp = spacy.load("fr_core_news_sm")

In [ ]:
# soit ligne par ligne

# lines = []
# with open("text1.txt", "r") as text1:
#     lines.append(text1.readline())


In [41]:
# Soit tout le texte d'un coup
fulltext = ""
with open("text1.txt", "r", encoding="utf-8") as text1:
    fulltext = text1.read()


doc = nlp(fulltext)

print("Nombre de token: ", len(doc))

Nombre de token:  1890


In [38]:
print("Graphe syntaxique de la première phrase du text1")
displacy.render(doc[:10], style="dep", jupyter=True)

Graphe syntaxique de la première phrase du text1


In [63]:
def print_token_info(token):
    print("Troisième token: ", token, f'({token.text}), index: ', token.i)
    print("Voisins: ", list(token.lefts), list(token.rights))

    print("Phrase: ", token.sent)
    print("Edges: ", token.left_edge, token.right_edge)
    print("Entity type: ", token.ent_type)
    print("Lemma: ", token.lemma, token.lemma_)
    print("Norm: ", token.norm, token.norm_)
    print("Shape: ", token.shape, token.shape_)
    print("Tag: ", token.tag, token.tag_)
    print("Sentiment: ", token.sentiment)
    print("-----")

print_token_info(doc[5])
# print_token_info(doc[2])
# print_token_info(doc[7])
print_token_info(doc[0])

Troisième token:  couché (couché), index:  5
Voisins:  [Longtemps, ,, je, me, suis] [heure, .]
Phrase:  Longtemps, je me suis couché de bonne heure.
Edges:  Longtemps .
Entity type:  0
Lemma:  10940960452429598961 coucher
Norm:  11184698726792784299 couché
Shape:  13110060611322374290 xxxx
Tag:  100 VERB
Sentiment:  0.0
-----
Troisième token:  Longtemps (Longtemps), index:  0
Voisins:  [] []
Phrase:  Longtemps, je me suis couché de bonne heure.
Edges:  Longtemps Longtemps
Entity type:  0
Lemma:  10169700273540668978 longtemps
Norm:  10169700273540668978 longtemps
Shape:  16072095006890171862 Xxxxx
Tag:  86 ADV
Sentiment:  0.0
-----


Insérer les données dans Neo4j

In [64]:
from neo4j import GraphDatabase

URI = "neo4j://localhost:7687"

with GraphDatabase.driver(URI, auth=None) as driver:
    driver.verify_connectivity()

Lister tous les verbes du corpus

Trouver les sujets d’un verbe donné : demander
Exemple : pour le verbe “manger”, afficher les tokens reliés par :DEP {label:"nsubj"}.

Trouver les objets directs associés aux verbes
→ relations :DEP {label:"dobj"}.

Compter combien de fois chaque dépendance grammaticale apparaît


Lister les adjectifs qui qualifient un nom donné : esprit, oiseau, train


Mesurer les verbes les plus utilisés dans le corpus (avec une requête Cypher)


Bonus:
Construire des couples Sujet–Verbe–Objet
Exemple : ("chat" - "mange" - "souris").

Bonus (dans l’ordre) : 
Calculez le graph de cooccurrence sous forme d’une matrice à partir du premier texte, importez ce graph dans neo4j puis mettez à jour ce graphe de cooccurrence  depuis le deuxième texte (il y a des mots en commun entre ces deux paragraphes vous vous en doutez) et mettez à jour la base neo4j.
Faites des recherches sur Neo4j Graph Data Science (GDS Library) et trouvez un algorithme de recherche de chemin le plus court, faites-en la description et expliquer comment l’utiliser avec cette librairie officielle
